In [3]:
#최상단에 import 를 추가한다.
#from google.colab.patches import cv2_imshow


In [6]:
import cv2
import numpy as np

# 웹캠/카메라 신호 받기
#VideoSignal = cv2.VideoCapture('http://www.utic.go.kr/view/map/cctvStream.jsp?cctvid=L020024&cctvname=%25EA%25B8%25B8%25EB%25B3%2591%25EC%259B%2590%25EC%2582%25AC%25EA%25B1%25B0%25EB%25A6%25AC&kind=N&cctvip=null&cctvch=null&id=L301&cctvpasswd=null&cctvport=null&minX=126.64385071435174&minY=37.41906266071739&maxX=126.79314754177085&maxY=37.485617497219316')
VideoSignal = cv2.VideoCapture(0)

# YOLO 가중치 파일과 CFG 파일 로드
YOLO_net = cv2.dnn.readNet("yolov2-tiny.weights","yolov2-tiny.cfg")

# YOLO NETWORK 재구성
classes = []
with open("yolo.names", "r") as f:                          
    classes = [line.strip() for line in f.readlines()]
layer_names = YOLO_net.getLayerNames()                                  # YOLO net의 모든 레이어 이름 받아오기                     
output_layers = [layer_names[i - 1] for i in YOLO_net.getUnconnectedOutLayers()] # 출력 레이어 이름 받아오기

In [7]:
while True:
    # 웹캠 프레임
    ret, frame = VideoSignal.read()
    h, w, c = frame.shape

    # YOLO 입력
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False) # (416, 416) : YOLO CNN 에서 사용되는 이미지 사이즈
    YOLO_net.setInput(blob)                 # 전체 신경망에 대한 입력 설정
    outs = YOLO_net.forward(output_layers)  # 출력층 이름을 매개변수로 전달하면 모든 출력 레이어에 대한 포워드 패스가 수행됩니다.

    class_ids = []          # 감지된 객체의 식별자
    confidences = []        # 신뢰도
    boxes = []              # 경계 박스 좌표

    for out in outs:

        for detection in out:

            scores = detection[5:]              # 감지확률
            class_id = np.argmax(scores)        # 가장 큰 값의 인덱스 가져오기
            confidence = scores[class_id]       # 신뢰도

            if confidence > 0.5:                    # 50% 이상의 신뢰도를 가진다면     
                # Object detected
                center_x = int(detection[0] * w)    # 경계상자 중심 좌표
                center_y = int(detection[1] * h)    # 경계상자 중심 좌표  
                dw = int(detection[2] * w)          # 경계상자 중심에서 경계상자까지의 폭 
                dh = int(detection[3] * h)          # 경계상자 중심에서 경계상자까지의 높이

                # Rectangle coordinate
                x = int(center_x - dw / 2)              # 경계상자의 x 좌표
                y = int(center_y - dh / 2)              # 경계상자의 y 좌표
                boxes.append([x, y, dw, dh])            # 경계상자의 좌표를 boxes 리스트에 추가
                confidences.append(float(confidence))   # 신뢰도를 confidences 리스트에 추가
                class_ids.append(class_id)              # 클래스 정보를 class_ids 리스트에 추가

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.45, 0.4)   # 겹치는 바인딩 박스 제거 (Non-Maximum Suppression)

    for i in range(len(boxes)):                     # 검출된 객체의 수 만큼 반복문 수행
        if i in indexes:                            # NMS 에 의해 선별된 경계상자만 그린다.
            x, y, w, h = boxes[i]                   # 경계상자의 좌표 가져오기
            label = str(classes[class_ids[i]])      # 객체 이름 가져오기
            score = confidences[i]                  # 신뢰도 가져오기

            # 경계상자와 클래스 정보 이미지에 입력
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 5)    # 경계상자 그리기
            cv2.putText(frame, label, (x, y - 20), cv2.FONT_ITALIC, 0.5,    # 객체 이름 출력
            (255, 255, 255), 1)

    cv2.imshow("frame", frame)           # 이미지 출력
    #cv2_imshow(frame)                    # Colab 에서 이미지 출력

    if cv2.waitKey(100) > 0:    # 아무 키나 누르면
        break

cv2.destroyAllWindows() # 열린 모든 창을 닫는다.

AttributeError: 'NoneType' object has no attribute 'shape'